In [10]:
import pandas as pd
import re
import os

# ---- Учитај фајлове ----
df1 = pd.read_csv("basic_imsbs.csv")  
df2 = pd.read_csv("learning_imsbs.csv")  

# ---- Извуци 4 броја из имена фајла ----
def extract_numbers(filename):
    # узимамо сва 4 броја из имена: n_m_k_index
    m = re.match(r".*?_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    else:
        return None, None, None, None

for df in [df1, df2]:
    df[['n','m','k','idx']] = df['file'].apply(lambda x: pd.Series(extract_numbers(x)))
    # kreiramo key po prva tri broja
    df['key'] = df[['n','m','k']].astype(str).agg('_'.join, axis=1)

# ---- Spoji podatke po key ----
merged = pd.merge(df1, df2, on='key', suffixes=('_1','_2'))

# ---- Agregiraj po prva tri broja (key) ----
agg = merged.groupby('key').agg({
    'time_1': 'mean',
    'quality_1': 'mean',
    'time_2': 'mean',
    'quality_2': 'mean'
}).reset_index()

# ---- Rezultat ----
print(agg)

# ---- Sačuvaj u CSV ----
agg.to_csv("merged_aggregated.csv", index=False)


         key    time_1  quality_1     time_2  quality_2
0   10_100_2  0.010758        8.6   0.016542        8.7
1   10_100_4  0.015646        6.2   0.021685        6.3
2   10_200_2  0.017391       10.3   0.020819       10.3
3   10_200_4  0.016767        6.2   0.023297        6.1
4   10_500_2  0.009166        9.5   0.045590        9.5
5   10_500_4  0.018688        6.1   0.032977        6.1
6    10_50_2  0.013251        9.1   0.013623        9.1
7    10_50_4  0.014968        6.0   0.017059        6.0
8    2_100_2  0.074699       72.8   0.169973       72.7
9    2_100_4  0.076843       61.7   0.164708       61.7
10   2_200_2  0.329666      136.9   1.079987      138.9
11   2_200_4  0.394792      125.1   0.978773      122.5
12   2_500_2  3.156137      272.1  14.580798      275.9
13   2_500_4  3.025999      299.3   9.984602      286.5
14    2_50_2  0.006316       37.7   0.009191       37.7
15    2_50_4  0.014924       30.1   0.023632       30.1
16   3_100_2  0.229338       58.5   0.421772    

In [11]:
agg[["time_1", "time_2", "quality_1", "quality_2" ]].mean()

time_1        0.769833
time_2        3.092795
quality_1    60.431250
quality_2    59.853125
dtype: float64